


# Assignment 5 on Natural Language Processing

## Date : 3rd Nov, 2020

### Instructor : Prof. Sudeshna Sarkar

### Teaching Assistants : Alapan Kuila, Aniruddha Roy, Anusha Potnuru, Uppada Vishnu

The central idea of this assignment is to explore various language models specifically LSTM based and transformer. We will explore how the size of the model effects the sequence generated. We will see both character based and word based models.



Please submit with outputs. Submissions without predicted outputs will be penalized.

# Word Based LSTM model

In [ ]:
import keras
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import LSTM
from keras.callbacks import ModelCheckpoint
from keras.layers import Embedding
from keras.utils import np_utils
import numpy
import os
import re
import string
import pandas as pd
import numpy as np
from nltk import word_tokenize
nltk.download('punkt')
from sklearn.model_selection import train_test_split
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


Do basic pre processing which includes lowering etc
Check the dataset and apply suitable preprocessing.

In [ ]:
# Load the data and preprocess data and store corpus in raw_text
!rm -r sample_data 
!wget 'https://www.dropbox.com/s/ogdasj2a375719a/corpus.txt?dl=1'
os.rename('./corpus.txt?dl=1', 'corpus.txt')
file = open("corpus.txt", "r")
original_text = file.read()

rm: cannot remove 'sample_data': No such file or directory
--2020-11-17 15:21:20--  https://www.dropbox.com/s/ogdasj2a375719a/corpus.txt?dl=1
Resolving www.dropbox.com (www.dropbox.com)... 162.125.65.1, 2620:100:6023:1::a27d:4301
Connecting to www.dropbox.com (www.dropbox.com)|162.125.65.1|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: /s/dl/ogdasj2a375719a/corpus.txt [following]
--2020-11-17 15:21:20--  https://www.dropbox.com/s/dl/ogdasj2a375719a/corpus.txt
Reusing existing connection to www.dropbox.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://uc11b86d420b346dcf6418b1af18.dl.dropboxusercontent.com/cd/0/get/BDag5O-_9SooJyMCMk7Fyo05R38DjESPOD7d-T3P105BmVgjjJtqtd-zyesPjwKXDioaEd3_vSJYszJ9ihMKQGvleycuIlt5KBTCtd9uL5ofpGZR-Qgcj-PL6dtnpAIejvo/file?dl=1# [following]
--2020-11-17 15:21:20--  https://uc11b86d420b346dcf6418b1af18.dl.dropboxusercontent.com/cd/0/get/BDag5O-_9SooJyMCMk7Fyo05R38DjESPOD7d-T3P105BmVgjjJtqtd-z

In [ ]:
def preprocess(raw_text):
    raw_text = raw_text.lower().replace('_', '')
    # split_text = raw_text.split('\n')
    # for i, line in enumerate(split_text):
    #     if line[:7] == 'CHAPTER':
    #         del split_text[i]
    # raw_text = '\n'.join(split_text)
    wordbased_raw_text = re.sub(r'[^\w\s]', '', raw_text)
    return wordbased_raw_text, raw_text

In [ ]:
word_raw_text, character_raw_text = preprocess(original_text)

In [ ]:
words = nltk.word_tokenize(word_raw_text)
print("No. of unique tokens for the word-based model = {}".format(len(set(words))))

No. of unique tokens for the word-based model = 2751


In [ ]:
# Hyperparameters of the model
vocab_size = 2751 # choose based on statistics
oov_tok = '<OOV>'
embedding_dim = 100
padding_type='post'
trunc_type='post'

In [ ]:
# tokenize sentences
tokenizer = Tokenizer(num_words = vocab_size, oov_token=oov_tok)
tokenizer.fit_on_texts([word_raw_text])
word_index = tokenizer.word_index

In [ ]:
seq_length = 50
tokens = tokenizer.texts_to_sequences([word_raw_text])[0]

In [ ]:
dataX = []
dataY = []

for i in range(0, len(tokens) - seq_length-1 , 1):
  seq_in = tokens[i:i + seq_length]
  seq_out = tokens[i + seq_length]

  if seq_out==1: #Skip samples where target word is OOV
    continue
    
  dataX.append(seq_in)
  dataY.append(seq_out)
 
N = len(dataX)
print ("Total training data size: ", N)

Total training data size:  26326


In [ ]:
X = numpy.array(dataX)

# one hot encode the output variable
y = numpy.array(dataY)
y = np_utils.to_categorical(dataY)

In [ ]:
# with embedding
model = keras.Sequential([
    keras.layers.Embedding(vocab_size, embedding_dim, input_length=seq_length),
    keras.layers.Bidirectional(keras.layers.LSTM(64)),
    keras.layers.Dense(vocab_size, activation='softmax')
])
# compile model
model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

# model summary
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_2 (Embedding)      (None, 50, 100)           275100    
_________________________________________________________________
bidirectional_1 (Bidirection (None, 128)               84480     
_________________________________________________________________
dense_1 (Dense)              (None, 2751)              354879    
Total params: 714,459
Trainable params: 714,459
Non-trainable params: 0
_________________________________________________________________


In [ ]:
# Use validation split of 0.2 while training
model.fit(X, y, epochs=12, batch_size=128, validation_split=0.2) 

Epoch 1/12
165/165 [==============================] - 7s 44ms/step - loss: 6.4338 - accuracy: 0.0544 - val_loss: 6.2618 - val_accuracy: 0.0849
Epoch 2/12
165/165 [==============================] - 7s 39ms/step - loss: 5.9567 - accuracy: 0.0561 - val_loss: 6.1848 - val_accuracy: 0.0849
Epoch 3/12
165/165 [==============================] - 7s 40ms/step - loss: 5.8237 - accuracy: 0.0619 - val_loss: 6.1638 - val_accuracy: 0.0934
Epoch 4/12
165/165 [==============================] - 7s 40ms/step - loss: 5.7180 - accuracy: 0.0684 - val_loss: 6.1695 - val_accuracy: 0.0946
Epoch 5/12
165/165 [==============================] - 7s 39ms/step - loss: 5.6258 - accuracy: 0.0756 - val_loss: 6.1667 - val_accuracy: 0.0929
Epoch 6/12
165/165 [==============================] - 7s 40ms/step - loss: 5.5251 - accuracy: 0.0843 - val_loss: 6.1441 - val_accuracy: 0.0949
Epoch 7/12
165/165 [==============================] - 7s 40ms/step - loss: 5.4204 - accuracy: 0.0943 - val_loss: 6.1232 - val_accuracy: 0.1001

In [ ]:
reverse_word_map = dict(map(reversed, tokenizer.word_index.items())) ## Create word to idx map using tokenizer.word_index

In [ ]:
# Complete the code to return next n words greedily
def next_tokens(input_str, n): 
		print ("Seed: \n",  input_str)
		result_string = ''
		for i in range(n):
				input_sequence = tokenizer.texts_to_sequences([input_str])[0]
				prediction = model.predict(input_sequence, verbose=0)
				predicted_word = reverse_word_map[numpy.argmax(prediction[0])]
				result_string = result_string + predicted_word + ' ' 
				input_str = input_str + ' ' + predicted_word
				input_str = ' '.join(input_str.split(' ')[1:])
		return result_string

In [ ]:
# pick a random seed
np.random.seed(42)

start = numpy.random.randint(0, len(dataX)-1)
pattern = dataX[start]
input_str = ' '.join([reverse_word_map[value] for value in pattern])

print('\nGenerated: \n' + next_tokens(input_str , 10))

Seed: 
 hare he denies it said the king leave out that part well at any rate the dormouse said the hatter went on looking anxiously round to see if he would deny it too but the dormouse denied nothing being fast asleep after that continued the hatter i cut some more

Generated: 
the know said was the queen alice realitythe duchess the 


In [ ]:
input_str = "The boy laughed at the fright he had caused. This time, the villagers left angrily. The third day, as the boy went up\
 the small hill, he suddenly saw a wolf attacking his sheep. He cried as hard as he could, “Wolf! Wolf! Wolf!”, but not \
 a single villager came to help him. The villagers thought that he was trying to fool them again and did not come to rescue \
 him or his sheep."

# Use first 50 tokens from given input_str as input.(Use tokenizer to split to take first 50)
print('\nGenerated: \n' + next_tokens(input_str, 10))

Seed: 
 The boy laughed at the fright he had caused. This time, the villagers left angrily. The third day, as the boy went up the small hill, he suddenly saw a wolf attacking his sheep. He cried as hard as he could, “Wolf! Wolf! Wolf!”, but not  a single villager came to help him. The villagers thought that he was trying to fool them again and did not come to rescue  him or his sheep.

Generated: 
queen not duchess the queen little know little the voice 


# Character based LSTM Model 1

In [ ]:
# User the preprocess data and create raw_text
raw_text = character_raw_text
# create mapping of unique characters to integers
chars = sorted(list(set(raw_text)))

int_to_char = dict((i, c) for i, c in enumerate(chars))
char_to_int = dict((c, i) for i, c in enumerate(chars))

In [ ]:
# Print the total characters and character vacob size
n_chars = len(raw_text)
n_vocab = len(chars)

print('Total number of characters = {}\nVocab size = {}'.format(n_chars, n_vocab))

Total number of characters = 142037
Vocab size = 45


In [ ]:
'''
Prepare dataset where the input is sequence of 100 characters and target is next character.
'''
seq_length = 100
dataX = []
dataY = []

for i in range(0, n_chars - seq_length, 1):

    input_seq = raw_text[i: i + seq_length]
    output_seq = raw_text[i + seq_length]
    dataX.append([char_to_int[char] for char in input_seq])
    dataY.append(char_to_int[output_seq])

n_patterns = len(dataX)
print ("Total Patterns: ", n_patterns)

Total Patterns:  141937


In [ ]:
# reshape X to be [samples, time steps, features]
X = numpy.reshape(dataX, (n_patterns, seq_length, 1))

# one hot encode the output variable
dataY = numpy.array(dataY)
y = np_utils.to_categorical(dataY)

In [ ]:
embedding_dim =100
max_length =100

In [ ]:
model = Sequential()
model.add(Embedding(n_vocab, embedding_dim, input_length=max_length))
model.add(LSTM(256))
model.add(Dropout(0.2))
model.add(Dense(y.shape[1], activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam')
model.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_3 (Embedding)      (None, 100, 100)          4500      
_________________________________________________________________
lstm_2 (LSTM)                (None, 256)               365568    
_________________________________________________________________
dropout (Dropout)            (None, 256)               0         
_________________________________________________________________
dense_2 (Dense)              (None, 45)                11565     
Total params: 381,633
Trainable params: 381,633
Non-trainable params: 0
_________________________________________________________________


In [ ]:
model.fit(X, y, epochs=20, batch_size=128)

Epoch 1/20
1109/1109 [==============================] - 46s 42ms/step - loss: 2.2060
Epoch 2/20
1109/1109 [==============================] - 46s 41ms/step - loss: 1.7016
Epoch 3/20
1109/1109 [==============================] - 46s 42ms/step - loss: 1.5040
Epoch 4/20
1109/1109 [==============================] - 47s 42ms/step - loss: 1.3830
Epoch 5/20
1109/1109 [==============================] - 46s 42ms/step - loss: 1.3012
Epoch 6/20
1109/1109 [==============================] - 46s 42ms/step - loss: 1.2383
Epoch 7/20
1109/1109 [==============================] - 46s 42ms/step - loss: 1.1870
Epoch 8/20
1109/1109 [==============================] - 46s 42ms/step - loss: 1.1443
Epoch 9/20
1109/1109 [==============================] - 46s 42ms/step - loss: 1.1045
Epoch 10/20
1109/1109 [==============================] - 46s 42ms/step - loss: 1.0692
Epoch 11/20
1109/1109 [==============================] - 46s 42ms/step - loss: 1.0369
Epoch 12/20
1109/1109 [==============================] - 46s 42

In [ ]:
#Get the generated string using the model.
def predict_next_100_chars(pattern, n):
    for i in range(n):
      reshaped = numpy.reshape(pattern, (1, len(pattern), 1))
      prediction = model.predict(reshaped, verbose=0)
      print (int_to_char[numpy.argmax(prediction)], end = '')  
      # seq_in = [int_to_char[value] for value in pattern]
      pattern.append(numpy.argmax(prediction))
      pattern = pattern[1:len(pattern)]

In [ ]:
#pick a random seed
np.random.seed(40)
start = numpy.random.randint(0, len(dataX)-1)
pattern = dataX[start]
input_str = ''.join([int_to_char[value] for value in pattern])
print("Seed:\n" + input_str + '\n')
print("\nGenerated string:\n")
predict_next_100_chars(pattern, 200)

Seed:
e often seen them at dinn—” she checked herself hastily.
“i don’t know where dinn may be,” said the dodo


Generated string:

 to the caterpillar and alice foldow it was the first to stair, and she was so much as she could not like the same thing as it was a little shriek, and was so much and see what was the first to stair,

In [ ]:
input_str = "The boy laughed at the fright he had caused. This time, the villagers left angrily. The third day, as the boy went up\
 the small hill, he suddenly saw a wolf attacking his sheep. He cried as hard as he could, “Wolf! Wolf! Wolf!”, but not \
 a single villager came to help him. The villagers thought that he was trying to fool them again and did not come to rescue \
 him or his sheep."
#Use the first 100 characters from given input_str as input and generate next 200 characters. 
input_str = input_str.lower()
input_string = ''
for each in input_str:
  if each in chars:
    if (len (input_string)<100):
      input_string += each
pattern = []
pattern.append([char_to_int[char] for char in input_string])
print ("\nGenerated:\n")
predict_next_n_chars(pattern[0], 200)


Generated:

nd the mock turtle said to herself “the moral of that is—‘the moral of that is—‘the moral of that is—‘the moral of that is—‘the moral of that is—‘the moral of that is—‘the moral of that is—‘the moral 

## Character based LSTM Model 2


In [ ]:
model1 = Sequential()
model1.add(Embedding(n_vocab, embedding_dim, input_length=max_length))
model1.add(LSTM(256, input_shape=(X.shape[1], embedding_dim),return_sequences=True))
model1.add(Dropout(0.2))
model1.add(LSTM(256))
model1.add(Dropout(0.2))
model1.add(Dense(y.shape[1], activation='softmax'))
model1.compile(loss='categorical_crossentropy', optimizer='adam')

In [ ]:
model1.fit(X, y, epochs=20, batch_size=64)

Epoch 1/20
2218/2218 [==============================] - 148s 67ms/step - loss: 2.0989
Epoch 2/20
2218/2218 [==============================] - 147s 66ms/step - loss: 1.5660
Epoch 3/20
2218/2218 [==============================] - 147s 66ms/step - loss: 1.3901
Epoch 4/20
2218/2218 [==============================] - 147s 66ms/step - loss: 1.2894
Epoch 5/20
2218/2218 [==============================] - 147s 66ms/step - loss: 1.2168
Epoch 6/20
2218/2218 [==============================] - 147s 66ms/step - loss: 1.1630
Epoch 7/20
2218/2218 [==============================] - 147s 66ms/step - loss: 1.1186
Epoch 8/20
2218/2218 [==============================] - 147s 66ms/step - loss: 1.0812
Epoch 9/20
2218/2218 [==============================] - 146s 66ms/step - loss: 1.0471
Epoch 10/20
2218/2218 [==============================] - 146s 66ms/step - loss: 1.0167
Epoch 11/20
2218/2218 [==============================] - 146s 66ms/step - loss: 0.9952
Epoch 12/20
2218/2218 [=============================

In [ ]:
# Generate the sequence similar to above methods. Get the generated string using the model.
def predict_next_n_chars(pattern, n):
    for i in range(n):
      x = numpy.reshape(pattern, (1, len(pattern), 1))
      prediction = model1.predict(x, verbose=0)
      print (int_to_char[numpy.argmax(prediction)], end = '') 
      seq_in = [int_to_char[value] for value in pattern]
      pattern.append(numpy.argmax(prediction))
      pattern = pattern[1:len(pattern)]

In [ ]:
#pick a random seed

start = numpy.random.randint(0, len(dataX)-1)
pattern = dataX[start]
input_str = ''.join([int_to_char[value] for value in pattern])
print ("Seed:",  input_str, sep = '\n\n')
print ("\nGenerated:\n")
predict_next_n_chars(pattern, 200)

Seed:

looking at the house, and wondering what to do next, when suddenly a footman in livery came running o

Generated:

n the subject.
“what a party was a little sisters of the court,” and she was a little shriek of the words “drink me,” said the king.
“i can go on the sea,” said the mock turtle.
“come on, i should thi

In [ ]:
input_str = "The boy laughed at the fright he had caused. This time, the villagers left angrily. The third day, as the boy went up\
 the small hill, he suddenly saw a wolf attacking his sheep. He cried as hard as he could, “Wolf! Wolf! Wolf!”, but not \
 a single villager came to help him. The villagers thought that he was trying to fool them again and did not come to rescue \
 him or his sheep."
#Use the first 100 characters from given input_str as input and generate next 200 characters. 
input_str = input_str.lower()
input_string = ''
for each in input_str:
  if each in chars:
    if (len (input_string)<100):
      input_string += each
pattern = []
pattern.append([char_to_int[char] for char in input_string])
print ("\nGenerated:\n")
predict_next_n_chars(pattern[0], 200)


Generated:

nd the mock turtle said to herself “the moral of that is—‘the moral of that is—‘the moral of that is—‘the moral of that is—‘the moral of that is—‘the moral of that is—‘the moral of that is—‘the moral 

# Questions

**Question:** What are your observations based on the model(all) outputs on train data(in domain) vs unseen data(out of domain) ?

**Answer:**
The models do not perform well on out of domain data. The performance on in-domain data is comparatively better.

**Question:** What did you observe in the outputs of char LSTM model1 vs char LSTM model2 ?

**Answer:**
Model 2, being more sophisticated in nature, is expected to perform better than Model 1 at coherence and grammar in the output sentence. Both models do not perform well, however.